In [22]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-07-22 21:45:24--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.1’

documents.json.1    100%[===================>] 642.90K  --.-KB/s    in 0.02s   

2024-07-22 21:45:25 (26.4 MB/s) - ‘documents.json.1’ saved [658332/658332]



In [23]:
!wget https://raw.githubusercontent.com/slv-ai/LLM-zoomcamp/main/Module1/minsearch.py

--2024-07-22 21:45:25--  https://raw.githubusercontent.com/slv-ai/LLM-zoomcamp/main/Module1/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2222 (2.2K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   2.17K  --.-KB/s    in 0s      

2024-07-22 21:45:25 (28.1 MB/s) - ‘minsearch.py.1’ saved [2222/2222]



In [24]:
import requests 
import minsearch
import json

In [25]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [26]:
documents = []

for course in docs_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [27]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [28]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [29]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [30]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [31]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [32]:
llm("i want to join the course, can i join it?")

" Joining a course typically involves following specific procedures rather than having open admission. You usually need to express your interest in joining and then proceed with any necessary steps required by the institution or organization offering the course. Here's what you might do:\n\n1. **Research** - First, find out if there are still available spots for new students on the desired course level (beginner, intermediate, advanced, etc.). This information can often be found directly from the educational institution’s website or by contacting their admissions office. \n\n2. **Express Interest** - If you're not already enrolled and it appears that there are still openings: write to them expressing your interest in joining the course, ensider email as a quicker form of communication while others may prefer applications via physical letters or online forms. Your tone should be professional yet enthusiastic about learning from their institution.\n\n3. **Meet Requirements** - Most cours

In [21]:
llm("course has started, can i still join it?")

" I'm sorry, but as an AI developed by Microsoft up until my last update in April 2decade or so (which does not include the current year), I don't have direct access to live courses. However, if a course you are interested in has started and is being taught through some educational platform like Coursera, edX, Udacity, or others, here is what your best steps would likely be:\n\n1. **Check for Enrollment**: First of all, look into whether the enrollment period is still open. This will vary depending on when exactly you are trying to join and how long courses typically run. Sometimes course providers extend periods after a program has started if there's significant demand or interest in continuing education beyond the initial term.\n\n2. **Promptly Enroll**: If it's still open for enrollment, go ahead as soon as possible to secure your spot. Popular courses can fill up quickly!\n\n3. **Reach Out Directly if Necessary**: In some cases, you may need help or guidance which is understandable